# Eat Safe, Love

## Notebook Set Up

In [2]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [6]:
# assign the collection to a variable
establishments = db['establishments']


## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [7]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
fields = {'BusniessName': 1, 'scores.Hygiene': 1}

# Use count_documents to display the number of documents in the result
document_count = establishments.count_documents(query)
print('Document Count:', document_count)
# Display the first document in the results using pprint
results = establishments.find(query, fields)
pprint(results[0])


Document Count: 41
{'_id': ObjectId('66ad4587b559f542eec6cada'), 'scores': {'Hygiene': 20}}


In [8]:
# Convert the result to a Pandas DataFrame
first_document_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print('Rows:', len(first_document_df))

# Display the first 10 rows of the DataFrame
first_document_df['HygieneScore'] = first_document_df['scores'].apply(lambda x: x['Hygiene'])
first_document_df.drop(columns=['scores'], inplace=True)
first_document_df.head(10)


Rows: 41


,_id,HygieneScore
0,66ad4587b559f542eec6cada,20
1,66ad4587b559f542eec6ce5e,20
2,66ad4587b559f542eec6d165,20
3,66ad4587b559f542eec6d35b,20
4,66ad4587b559f542eec6d35d,20
5,66ad4587b559f542eec6dd02,20
6,66ad4587b559f542eec6df2e,20
7,66ad4587b559f542eec6f44a,20
8,66ad4587b559f542eec6f891,20
9,66ad4587b559f542eec700a2,20


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [9]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex': 'London'}, 'RatingValue': {'$gte': 4}}
fields = {'BusinessName': 1, 'LocalAuthorityName': 1, 'RatingValue': 1}

# Use count_documents to display the number of documents in the result
local_counts = establishments.count_documents(query)
print('London Count:', local_counts)

# Display the first document in the results using pprint
local_first_document = establishments.find(query, fields).limit(1)
pprint(local_first_document[0])


London Count: 33
{'BusinessName': "Charlie's",
 'LocalAuthorityName': 'City of London Corporation',
 'RatingValue': 4,
 '_id': ObjectId('66ad4587b559f542eec6e4f5')}


In [10]:
# Convert the result to a Pandas DataFrame
local_documents = list(establishments.find(query, fields))
local_document_df = pd.DataFrame(local_documents)

# Display the number of rows in the DataFrame
print('Rows:', len(local_document_df))

# Display the first 10 rows of the DataFrame
local_document_df.head(10)

Rows: 33


,_id,BusinessName,RatingValue,LocalAuthorityName
0,66ad4587b559f542eec6e4f5,Charlie's,4,City of London Corporation
1,66ad4587b559f542eec6e81b,Mv City Cruises Erasmus,5,City of London Corporation
2,66ad4587b559f542eec6f366,Benfleet Motor Yacht Club,4,City of London Corporation
3,66ad4587b559f542eec70167,Coombs Catering t/a The Lock and Key,5,City of London Corporation
4,66ad4587b559f542eec70168,Tilbury Seafarers Centre,5,City of London Corporation
5,66ad4587b559f542eec7086f,Mv Valulla,5,City of London Corporation
6,66ad4588b559f542eec7297b,Tereza Joanne,5,City of London Corporation
7,66ad4588b559f542eec72d44,Brick Lane Brews,4,City of London Corporation
8,66ad4588b559f542eec72d5f,The Nuance Group (UK) Limited,5,City of London Corporation
9,66ad4588b559f542eec72d60,WH Smith,5,City of London Corporation


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [11]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {
    'geocode.latitude': {'$gte': latitude - degree_search, '$lte': latitude + degree_search},
    'geocode.longitude': {'$gte': longitude - degree_search, '$lte': longitude + degree_search},
    'RatingValue': 5
}

sort = [('scores.Hygiene', 1)] 
limit = 5

# Print the results
results = establishments.find(query).sort(sort).limit(limit)
for result in results:
    print(result)


{'_id': ObjectId('66ad4588b559f542eec72337'), 'FHRSID': 694609, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000116619', 'BusinessName': 'Volunteer', 'BusinessType': 'Pub/bar/nightclub', 'BusinessTypeID': 7843, 'AddressLine1': '130 - 132 Plumstead High Street', 'AddressLine2': '', 'AddressLine3': 'Plumstead', 'AddressLine4': 'Greenwich', 'PostCode': 'SE18 1JQ', 'Phone': '', 'RatingValue': 5, 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2019-08-05T00:00:00', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0}, 'SchemeType': 'FHRS', 'geocode': {'longitude': 0.09208, 'latitude': 51.4873437, 'logitude': None}, 'RightToReply': '', 'Distance': 4646.965634598608, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returnco

In [12]:
# Convert result to Pandas DataFrame
updated_df = pd.DataFrame(establishments.find(query).sort(sort).limit(limit))
updated_df.head()


,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links,geocod
0,66ad4588b559f542eec72337,694609,0,PI/000116619,Volunteer,Pub/bar/nightclub,7843,130 - 132 Plumstead High Street,,Plumstead,...,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.09208, 'latitude': 51.4873437,...",,4646.965635,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...","{'longitude': 0.09208, 'latitude': 51.4873437}"
1,66ad4588b559f542eec72352,694478,0,PI/000086506,Atlantic Fish Bar,Takeaway/sandwich shop,7844,35 Lakedale Road,,Plumstead,...,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.0912164, 'latitude': 51.486729...",,4646.974612,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...","{'longitude': 0.0912164, 'latitude': 51.4867296}"
2,66ad4588b559f542eec72354,909912,0,PI/000201100,Lumbini Grocery Ltd T/A Al-Iman,Retailers - other,4613,17 Lakedale Road,,Plumstead,...,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.0916256, 'latitude': 51.487163...",,4646.975587,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...","{'longitude': 0.0916256, 'latitude': 51.4871632}"
3,66ad4588b559f542eec7230a,695223,0,PI/000178842,Iceland,Retailers - supermarkets/hypermarkets,7840,144 - 146 Plumstead High Street,,Plumstead,...,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.0924199968576431, 'latitude': ...",,4646.946071,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...","{'longitude': 0.0924199968576431, 'latitude': ..."
4,66ad4588b559f542eec7231a,1380578,0,14425,Howe and Co Fish and Chips - Van 17,Mobile caterer,7846,Restaurant And Premises 107A Plumstead High St...,,Plumstead,...,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.0925370007753372, 'latitude': ...",,4646.955931,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...","{'longitude': 0.0925370007753372, 'latitude': ..."


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [13]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
hygiene_score = {"$match": {"scores.Hygiene": 0}}
groups = {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum" : 1}}}
sorts = {"$sort": {"count": -1}}

pipeline = [hygiene_score, groups, sorts]
results = establishments.aggregate(pipeline)

# Print the number of documents in the result
results_list = list(results)
print('Total Documents:', len(results_list))

# Print the first 10 results
pprint(results_list[:10])


Total Documents: 55
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [14]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results_list)

# Display the number of rows in the DataFrame
print('Rows:', len(df))

# Display the first 10 rows of the DataFrame
df.head(10)


Rows: 55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
